In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

# Data application ( prices + features)

In [8]:
aapl_with_features = pd.read_csv('../data/all_data/all_AAPL_data.csv')
googl_with_features = pd.read_csv('../data/all_data/all_GOOGL_data.csv')
msft_with_features = pd.read_csv('../data/all_data/all_MSFT_data.csv')

shares = ["AAPL", "GOOGL", "MSFT"]
for share, df in zip(shares, [aapl_with_features, googl_with_features, msft_with_features]):
    print(share)
    print(df.head()) 

print(shares)

AAPL
    index        DATE  Close_AAPL  High_AAPL  Low_AAPL  Open_AAPL  \
0  1508.0  2006-01-04    2.241830   2.241830  2.166852   2.170751   
1  1509.0  2006-01-05    2.248428   2.278719  2.234332   2.253227   
2  1510.0  2006-01-06    2.230734   2.246329  2.211840   2.244230   
3  1511.0  2006-01-09    2.288316   2.300313  2.235832   2.256826   
4  1512.0  2006-01-10    2.280819   2.315310  2.271523   2.301214   

   Volume_AAPL     RSI_14  MACD_12_26_9  MACDh_12_26_9  ...  Close_MSFT  \
0  807234400.0  63.192529      0.052154      -0.013956  ...   18.715057   
1  619603600.0  63.759171      0.053152      -0.010367  ...   18.805702   
2  449422400.0  61.045151      0.051916      -0.009282  ...   18.819654   
3  704457600.0  66.102053      0.054950      -0.004998  ...   18.763872   
4  675040800.0  64.920429      0.056103      -0.003076  ...   18.729006   

   High_MSFT   Low_MSFT  Open_MSFT  Volume_MSFT  Close_GOOGL  High_GOOGL  \
0  18.826622  18.199069  18.303661   79973000.0    10

# Target

In [9]:
def create_target_5d_threshold(
    df,
    close_col,
    horizon=5,
    threshold=0.01
):
    df = df.copy()

    future_return = (
        df[close_col].shift(-horizon) / df[close_col] - 1
    )

    df["Target"] = (future_return > threshold).astype(int)

    df = df.iloc[:-horizon]

    return df

for share, stock_data in zip(shares, [aapl_with_features, googl_with_features, msft_with_features]):
    create_target_5d_threshold(
        stock_data,
        close_col=f"Close_{share}",
        horizon=5,
        threshold=0.01
    )


# Training baseline model (Logistic regression) + feature importance

In [10]:

from os import pipe

result_dict = {}

def train_baseline_model(data, share_name):

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(max_iter=10000, random_state=42))
    ])

    features = data.drop(columns=['index','Target', 'DATE', f'Close_{share_name}'])
    features = features.dropna()

    target = data.loc[features.index, 'Target']
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]
    # plt.hist(y_proba, bins=50)
    # plt.show()
    
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    print(f"Results for {share_name}:")
    print("Accuracy:", acc)
    print("Precision:", precision)
    print("Recall:", recall)
    print("ROC AUC Score:", roc_auc)
    
    result_dict[share_name] = [acc, precision, recall, roc_auc]

print(shares)
for share, df in zip(shares, [aapl_with_features, googl_with_features, msft_with_features]):
    train_baseline_model(df, share)

def train_feature_impact(data, share_name):

    features = data.drop(columns=['index','Target', 'DATE', f'Close_{share_name}'])
    features = features.dropna()

    target = data.loc[features.index, 'Target']

    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=10000, random_state=42))
    ])

    pipe.fit(X_train, y_train)
    pipe.predict(X_test)

    coefficients = pipe.named_steps["clf"].coef_[0]
    feature_names = X_train.columns
    feature_importance = pd.DataFrame({
        "Feature": feature_names,
        "Coefficient": coefficients,
        "Abs_Coefficient": np.abs(coefficients),
        "Odds_Ratio": np.exp(coefficients)
    }).sort_values(by="Abs_Coefficient", ascending=False)

    print(feature_importance)

 
for share, df in zip(shares, [aapl_with_features, googl_with_features, msft_with_features]):
    train_feature_impact(df, share)



['AAPL', 'GOOGL', 'MSFT']
Results for AAPL:
Accuracy: 0.5119047619047619
Precision: 0.5317073170731708
Recall: 0.615819209039548
ROC AUC Score: 0.5045304338556658
Results for GOOGL:
Accuracy: 0.4623015873015873
Precision: 0.48091603053435117
Recall: 0.11731843575418995
ROC AUC Score: 0.52466521960882
Results for MSFT:
Accuracy: 0.5357142857142857
Precision: 0.5310853530031612
Recall: 0.9563567362428842
ROC AUC Score: 0.5448445087913778
           Feature  Coefficient  Abs_Coefficient  Odds_Ratio
22        Open_VIX     0.377664         0.377664    1.458873
11  BBU_20_2.0_2.0    -0.352166         0.352166    0.703164
1         Low_AAPL    -0.322279         0.322279    0.724496
33      High_GOOGL    -0.291139         0.291139    0.747412
27      Close_MSFT     0.247262         0.247262    1.280514
21         Low_VIX    -0.240574         0.240574    0.786176
20        High_VIX    -0.240037         0.240037    0.786599
9   BBL_20_2.0_2.0     0.233633         0.233633    1.263180
8          

In [11]:
df_to_save = pd.DataFrame.from_dict(
    result_dict, 
    orient='index',
    columns=['Accuracy', 'Precision', 'Recall', 'ROC AUC']
)
print(df_to_save)

df_to_save.to_csv('../models_result/LR_results.csv')

       Accuracy  Precision    Recall   ROC AUC
AAPL   0.511905   0.531707  0.615819  0.504530
GOOGL  0.462302   0.480916  0.117318  0.524665
MSFT   0.535714   0.531085  0.956357  0.544845
